In [ ]:
%%html
<span style="color:red; font-family:Helvetica Neue, Helvetica, Arial, sans-serif; font-size:2em;">An Exception was encountered at 'In [4]'.</span>

An Exception was encountered at 'In [4]'.

In [1]:
# Parameters
load_path = "data/scraped_gw_reviews.csv"
save_path = "data/cleaned_gw_reviews.csv"


# Verify and Clean

#### Evan Yathon
#### 

This notebook is intended to be run with papermill from the project root.

The purpose of this notebook is to take a first look at the data, clean it where necessary and ensure that all reviews in the original given dataset (`data/given_4U_reviews.txt`) all exist in my newly scraped dataset.

Usage:

`papermill src/ipynbs/verify_and_clean.ipynb src/verify_and_clean_ran.ipynb -p load_path data/scraped_gw_reviews.csv -p save_path data/cleaned_gw_reviews.csv`

In [2]:
#parameters section for Papermill

load_path = "../../data/scraped_gw_reviews.csv"
save_path = "../../data/cleaned_gw_reviews.csv"

This notebook will load the scraped data, verify that all of the reviews scraped are in the original dataset, and then clean up the columns for use in analyis later.

In [3]:
# loading packages

# utils
import pandas as pd
import time
import numpy as np

# plotting
import seaborn as sns
import matplotlib.pyplot as plt

In [4]:
# ensure the dates are parsed correctly with parse_dates argument
reviews = pd.read_csv(load_path, parse_dates = ["date_of_review", "date_flown"])

FileNotFoundError: [Errno 2] File b'../../data/scraped_gw_reviews.csv' does not exist: b'../../data/scraped_gw_reviews.csv'

In [ ]:
reviews.head(50)

In [ ]:
# check what column types we are working with
reviews.dtypes

In [ ]:
# check to see how many missing values there are for each column
print("Percentage of missing values for each column: \n")
(reviews.isna().sum()/reviews.shape[0])*100

It seems that most reviews are missing the majority of `n_user_reviews`, `aircraft`, `traveller_type`, `route`, `date_flown`, and `ground_service_rating`.

Trying to infer things from these columns will likely be not very useful as we only have 146 samples to use.  

#### Verifying that all reviews are in original dataset

In [ ]:
# load the original dataset

f = open("../../data/given_4U_reviews.txt")
orig_reviews = f.read()
f.close()

In [ ]:
# check to make sure that all our extracted reviews are in the original dataset
#
# note that the last review in the given reviews was from 16th October 2015, 
# so there will be some extra reviews
#
# print the date and title of the review if it is not found in the original reviews

for count, review in enumerate(reviews["review_text"]):
    if review not in orig_reviews:
        print(reviews["date_of_review"][count]," : ", reviews["title"][count],"\n")

Most of these reviews are after 16th October 2016, so we know that it is mostly good.  But there are 7 reviews at the bottom that should be showing up as in the scraped dataset.  Let's look at the `2015-09-09 00:00:00  :  "staff friendly - food terrible"` entry.

In [ ]:
# the original review copied from the given text file
review_from_given_txt = """"staff friendly - food terrible"
Jay Martin (Germany) 9th September 2015
7
Easy Check-In at Düsseldorf. The Germanwings flight was on time. Seats good for a low-cost airline. The food was terrible and expensive. The normal Ticket price excludes luggage, and for including luggage check-in you have to pay extra money. Staff were friendly.
Type Of Traveller	Solo Leisure
Cabin Flown	Economy
Route	Düsseldorf to London
Date Flown	August 2015
Seat Comfort	12345
Cabin Staff Service	12345
Food & Beverages	12345
Inflight Entertainment	12345
Ground Service	12345
Wifi & Connectivity	12345
Value For Money	12345
Recommended	yes"""
print(review_from_given_txt)

In [ ]:
# get the review from the scraped dataset
reviews["review_text"][reviews["title"].str.contains("staff friendly - food terrible")].values[0]

The two reviews are identical, but there is likely some hidden formatting messing things up.  

I verified that the other 6 reviews were also the same, and with that am confident that my dataset has the same content with additional information compared to the provided one.

#### Date Differences

Check to see what the difference in between the date flown and date reviewed to see if the review date is close enough to be used for date flown.

In [ ]:
# figure out the time difference in days, removing any NA values
time_difference = (reviews["date_of_review"] - reviews["date_flown"]).dt.days.dropna().values

In [ ]:
sns.distplot(time_difference, kde = False)
plt.title("Difference Between Date Reviewed\n and Date Flown in Days")

In [ ]:
np.quantile(time_difference, 0.8)

It seems that 80% of reviews are posted within 50 days, and since we are missing most of the `date_flown` columns, I'll assume that the date flown is close to the date reviewed.

#### Data Cleaning

Some data cleaning is now necessary.  Here is the plan:

- Isolate the reviewer country
- *Split `route` into origin and destination* (not crucial due to large amount of NaNs)

##### Isolate the reviewer country

Each entry has something along the lines of

```
\n\nRalf-Peter Weber (Gibraltar) 31st August 2015
```

With the country consistently in brackets.  We can use regex to isolate the country name.

Source:
https://stackoverflow.com/questions/54077038/use-regular-expression-to-extract-elements-from-a-pandas-data-frame

In [ ]:
# pattern matches any characters between brackets
pattern = "\((.*)\)"
# extract items that match the pattern in reviewer_countries
countries = reviews["reviewer_country"].str.findall(pattern).str[0]

# replace reviewer country with extracted countries
reviews["reviewer_country"] = countries

Since `route` has ~80% of it's data missing, in interest of time I won't parse the column into origin and destination.  If there were more data I would split the string on the word "to".

In [ ]:
reviews.to_csv(save_path, index = False)